<a href="https://colab.research.google.com/github/asu-trans-ai-lab/PA-DTA/blob/main/PA_DTA_DNL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import logging
import torch
import torch.nn.functional as F

# -------------------------
# Setup logging
# -------------------------
logging.basicConfig(
    level=logging.INFO,
    filename='flowthrough.log',
    filemode='w',
    format='%(asctime)s [%(levelname)s] %(message)s'
)
logger = logging.getLogger()

# -------------------------
# 1) Load & pad observed Q
# -------------------------
raw_Q = {
    'a': [
        4.015100057,14.89955676,17.58747607,31.55234724,53.62932332,24.90501662,23.42980658,34.07206404,
        29.86934248,18.68268658,24.44440088,10.03088594,43.39953279,28.57850658,38.40417973,39.02527158,
        32.27089766,46.22993711,35.95010947,44.91370014,54.87553573,44.32281976,30.01943968,97.06285496,
        66.0427673,59.49307137,73.15263459,62.91030402,61.80472875,36.29623161,46.15488851,28.63392708,
        25.89176672,49.62036794,69.90240954,35.56149008,43.31376297,59.11520429,46.59017038,21.46389937,
        56.34417909,39.62566038,59.85125786,71.77375123,104.8371201,67.77465805,81.53279817,81.35268153,
        73.80641549,96.33511097,47.76622608,60.41412236,44.62890032,9.586039562,2.076687266
    ],
    'b': [
        1.67839196,7.554140127,6.542372881,5.760869565,6.285714286,9.0,12.97931034,11.2173913,
        26.35294118,24.30769231,14.27272727,15.34375,13.10791367,16.95327103,16.04878049,19.2038835,
        34.96,44.96551724,53.53488372,91.77777778,59.04347826,63.47826087,95.93548387,56.38461538,
        70.0,67.9,72.05714286,75.57142857,78.0,120.7272727,139.5,99.07692308,
        120.88,124.8571429,111.76,78.22857143,107.5652174,87.33333333,158.2352941,122.0,
        146.4210526,117.5,83.28571429,134.5714286,89.69230769,73.77142857,68.0,98.8,
        75.53846154,47.04347826,33.83098592,25.31034483,13.11111111,26.90909091,15.94495413,2.5,
        3.962264151,1.789473684,2.112359551,5.034482759
    ],
    'c': [
        5.355932203,6.125,20.96350365,14.80851064,20.09022556,26.76595745,34.92307692,17.4351145,
        29.36,44.89552239,39.05,32.56470588,24.63366337,34.25,30.57894737,31.85714286,
        39.05882353,28.05479452,47.13043478,72.0,52.11764706,51.40425532,57.46341463,72.4,
        63.90243902,66.71428571,52.7,55.35135135,74.0,86.35294118,81.0,55.0,
        63.41176471,53.3,51.57894737,78.56,61.80645161,78.95652174,75.42857143,69.14285714,
        78.36363636,79.07692308,45.63636364,68.0,64.27586207,38.10909091,33.68421053,38.0,
        76.8,25.33333333,41.0,41.76271186,23.03797468,36.38095238,30.51851852,32.96,
        32.48,31.55,10.24460432
    ],
    'd': [
        2.908256881,24.85840708,35.0,22.65217391,24.23966942,21.47619048,11.19354839,20.93442623,
        12.76470588,36.61290323,41.45,34.73913043,34.72093023,29.53333333,35.68571429,43.58823529,
        59.85714286,44.0,42.25373134,44.29411765,45.98461538,50.75,42.92857143,51.94736842,59.0,
        44.0,38.36,41.0,51.90909091,41.0,34.8125,37.68085106,50.19148936,43.72727273,47.26086957,
        44.42857143,49.81632653,53.0,41.0,39.5,37.66666667,37.72727273,44.0,31.14925373,38.65853659,
        35.15789474,28.52631579,38.53846154,31.93333333,29.26373626,21.54054054,3.082840237
    ]
}
T_max = max(len(v) for v in raw_Q.values())
Q_obs = torch.stack([
    torch.tensor(raw_Q[k] + [0.0]*(T_max - len(raw_Q[k])), dtype=torch.float32)
    for k in ['a','b','c','d']
])

# -------------------------
# 2) Simple inflow proxy λ(t)
# -------------------------
lambda_obs = F.relu(Q_obs[:,1:] - Q_obs[:,:-1])
lambda_obs = torch.cat([lambda_obs, torch.zeros(Q_obs.shape[0], 1)], dim=1)

# -------------------------
# 3) Path‐arc incidence
# -------------------------
incidence = torch.tensor([[1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0]], dtype=torch.float32)
print("Path-to-Arc Incidence Matrix:")
print(incidence)
logger.info(f"Incidence matrix:\n{incidence}")

# -------------------------
# 4) Pipeline modules
# -------------------------
class PreProcessing:
    def estimate_phases(self, Q):
        nz = Q > 0
        t0 = nz.float().argmax(dim=1)
        t3_rev = nz.flip(dims=[1]).float().argmax(dim=1)
        t3 = Q.shape[1] - 1 - t3_rev
        return t0, t3

    def fit_inflow(self, lam, t0):
        lam = F.relu(lam)
        arcs, T = lam.shape
        t = torch.arange(T, device=lam.device, dtype=lam.dtype).unsqueeze(0)
        dt = t - t0.unsqueeze(1)
        A = torch.stack([dt**2, dt, torch.ones_like(dt)], dim=2)
        coeffs = torch.stack([
            torch.linalg.lstsq(A[i], lam[i].unsqueeze(1)).solution.squeeze()
            for i in range(arcs)
        ], dim=0)
        a, b, c = coeffs[:,0:1], coeffs[:,1:2], coeffs[:,2:3]
        lam_fit = F.relu(a*dt**2 + b*dt + c)
        beta = F.relu(coeffs[:,0])
        return beta, lam_fit

class SimulateLoad(torch.nn.Module):
    def __init__(self, capacity):
        super().__init__()
        self.capacity = capacity
    def forward(self, U):
        U = F.relu(U)
        arcs, T = U.shape
        Q = torch.zeros_like(U)
        X = torch.zeros_like(U)
        for t in range(T):
            out = F.relu(torch.minimum(Q[:,t], self.capacity[:,t]))
            X[:,t] = out
            if t+1 < T:
                Q[:,t+1] = F.relu(Q[:,t] + U[:,t] - out)
        return X, Q

class EstimatePhases:
    def __call__(self, Q):
        nz = Q > 0
        t0 = nz.float().argmax(dim=1)
        t3_rev = nz.flip(dims=[1]).float().argmax(dim=1)
        t3 = Q.shape[1] - 1 - t3_rev
        return t0, t3

class EvolveState(torch.nn.Module):
    def __init__(self, incidence):
        super().__init__()
        self.register_buffer('inc', incidence)
    def forward(self, path_flows):
        pf = F.relu(path_flows)
        return F.relu(self.inc.T @ pf)

class DelayRepresentation:
    def __init__(self, beta, t0, t3, mu, tau_free):
        self.beta = beta.unsqueeze(1)
        self.t0 = t0.unsqueeze(1)
        self.t3 = t3.unsqueeze(1)
        self.mu = mu.unsqueeze(1)
        self.tau_free = tau_free.unsqueeze(1)
    def __call__(self, T):
        t = torch.arange(T, device=self.beta.device, dtype=self.beta.dtype).unsqueeze(0)
        dt0 = t - self.t0
        dt3 = self.t3 - t
        W = self.beta * dt0**2 * dt3 / self.mu
        tau = self.tau_free + W
        return W, tau

class PathTravelTime(torch.nn.Module):
    def __init__(self, paths):
        super().__init__()
        self.paths = paths
    def forward(self, tau, entry_times):
        P, T = entry_times.shape
        costs = torch.zeros_like(entry_times)
        for p, path in enumerate(self.paths):
            theta = entry_times[p].clone()
            for a in path:
                floor_i = torch.clamp(theta.floor().long(), 0, T-2)
                alpha = theta - floor_i.float()
                tau_a = tau[a]
                tau_hat = tau_a[floor_i]*(1-alpha) + tau_a[floor_i+1]*alpha
                theta = theta + tau_hat
            costs[p] = F.relu(theta - entry_times[p])
        return costs

class FlowthroughTensor(torch.nn.Module):
    def __init__(self, Q_obs, incidence, tau_free):
        super().__init__()
        arcs, T = Q_obs.shape
        cap = Q_obs.max(dim=1).values
        cap_time = cap.unsqueeze(1).repeat(1, T)
        self.cap = cap
        self.pre = PreProcessing()
        self.sim = SimulateLoad(cap_time)
        self.ep = EstimatePhases()
        self.state = EvolveState(incidence)
        self.path_cost = PathTravelTime(paths=[[0,1],[2,3]])
        self.tau_free = tau_free
    def forward(self, lam_obs, Q_obs, path_flows):
        print("=== Input Confirmation ===")
        print(f"Incidence matrix:\n{incidence}")
        print(f"Q_obs shape: {Q_obs.shape}, lambda_obs: {lambda_obs.shape}")
        t0, t3 = self.pre.estimate_phases(Q_obs)
        beta, lam_fit = self.pre.fit_inflow(lam_obs, t0)
        print("=== PreProcessing ===")
        print(f"t0: {t0.tolist()}, beta: {beta.tolist()}")
        X, Qx = self.sim(lam_fit)
        print("=== Model X ===")
        print(f"X[:,:5]:\n{X[:,:5]}")
        t0x, t3x = self.ep(Qx)
        print("=== Model Y ===")
        print(f"t0x: {t0x.tolist()}, t3x: {t3x.tolist()}")
        delay = DelayRepresentation(beta, t0x, t3x, mu=self.cap, tau_free=self.tau_free)
        W, tau = delay(Qx.shape[1])
        print("=== DelayRepresentation τ[:,:5] ===")
        print(tau[:,:5])
        entry = torch.arange(Qx.shape[1]).unsqueeze(0).repeat(path_flows.shape[0],1)
        path_costs = self.path_cost(tau, entry)
        print("=== PathTravelTime costs[:,:5] ===")
        print(path_costs[:,:5])
        arc_inflows = self.state(path_flows)
        print("=== Model Z ===")
        print(arc_inflows[:,:5])
        loss = F.mse_loss(X, Q_obs)
        print(f"Loss (MSE between X and Q_obs): {loss.item()}")
        return {'X': X, 'Qx': Qx, 'tau': tau, 'path_costs': path_costs, 'arc_inflows': arc_inflows, 'loss': loss}

if __name__ == '__main__':
    tau_free = torch.tensor([1.0,1.0,1.0,1.0], dtype=torch.float32)
    path_flows = torch.rand(2, T_max)
    model = FlowthroughTensor(Q_obs, incidence, tau_free)
    out = model(lambda_obs, Q_obs, path_flows)
    print("=== Final Outputs ===")
    for k, v in out.items(): print(k, (v.shape if hasattr(v, 'shape') else v))


Path-to-Arc Incidence Matrix:
tensor([[1., 1., 0., 0.],
        [0., 0., 1., 1.]])
=== Input Confirmation ===
Incidence matrix:
tensor([[1., 1., 0., 0.],
        [0., 0., 1., 1.]])
Q_obs shape: torch.Size([4, 60]), lambda_obs: torch.Size([4, 60])
=== PreProcessing ===
t0: [0, 0, 0, 0], beta: [0.0, 0.0, 0.0, 0.0007945054676383734]
=== Model X ===
X[:,:5]:
tensor([[0.0000, 4.8545, 5.2528, 5.6352, 6.0015],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5331],
        [0.0000, 4.2649, 4.4764, 4.6795, 4.8742],
        [0.0000, 7.0833, 6.9209, 6.7601, 6.6008]])
=== Model Y ===
t0x: [1, 4, 1, 1], t3x: [59, 58, 59, 59]
=== DelayRepresentation τ[:,:5] ===
tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0008, 1.0000, 1.0008, 1.0030, 1.0066]])
=== PathTravelTime costs[:,:5] ===
tensor([[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]])
=== Model Z ===
tensor([[0.4108, 0.3969, 0.4736, 0.4911, 0.265